In [1]:
import numpy as np


In [2]:
input = '''>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
'''

# with open('input') as f:
#     input = f.read()

input = input.splitlines()[0]


In [3]:
jet = list(input)

NB_JETS = len(jet)


In [4]:
nextJet = 0
def get_next_jet():
    global nextJet
    result = jet[nextJet]
    nextJet += 1
    nextJet %= NB_JETS
    
    if result == '>':
        return 1
    elif result == '<':
        return -1
    else:
        assert False
    return 0


In [5]:
rockShapes = np.full((5, 4, 4), 0)  # 5 rock types with 4x4 each
rockShapes[0] = np.array([  # rock shapes are defined upside-down
    [1, 1, 1, 1],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])
rockShapes[1] = np.array([
    [0, 1, 0, 0],
    [1, 1, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
])
rockShapes[2] = np.array([
    [1, 1, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0]
])
rockShapes[3] = np.array([
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0]
])
rockShapes[4] = np.array([
    [1, 1, 0, 0],
    [1, 1, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])

nextShape = 0
NB_SHAPES = len(rockShapes)


def get_next_shape():
    global nextShape
    result = rockShapes[nextShape]
    nextShape += 1
    nextShape %= NB_SHAPES
    return result


In [6]:
CAVE_WIDTH = 7
floor = [1] * CAVE_WIDTH
level = [0] * CAVE_WIDTH

cave = np.array(floor)
cave.resize(2, CAVE_WIDTH)

heightRocks = 0  # floor is level 0


In [7]:
def is_position_valid(cave, rock, col, row):
    cavePart = cave[row:row + 4, col:col + 4]
    while cavePart.shape[1] < 4:
        cavePart = np.column_stack([cavePart, np.array([1] * 4)])
    return (cavePart + rock).max() <= 1


def place_rock(cave, rock, col, row):
    cavePart = cave[row:row + 4,col:col + 4]
    cave[row:row + 4, col:col + 4] = cavePart | rock[:cavePart.shape[0], :cavePart.shape[1]]


In [8]:
NB_FALLINGROCKS = 2022


for nbRocks in range(NB_FALLINGROCKS):
    # extend cave height
    if cave.shape[0] < heightRocks + 3 + 4 + 2:
        cave.resize(heightRocks + 3 + 4 + 2, CAVE_WIDTH, refcheck=False)
    
    currX = 2  # two units away from the left wall
    currY = heightRocks + 4  # three units above the highest rock in the cave
    currRock = get_next_shape()
#     print ('Rock created at position: ' + str((currX, currY)))
    
    while True:
        # jet pushes rock
        nextX = currX + get_next_jet()
        
        if is_position_valid(cave, currRock, nextX, currY):
            currX = nextX
#             print ('New rock position: ' + str((currX, currY)))
        
        # fall down one unit
        nextY = currY - 1
        
        if is_position_valid(cave, currRock, currX, nextY):
            currY = nextY
#             print ('New rock position: ' + str((currX, currY)))
            if currY < 0:
                break
            else:
                continue
        else:
#             print ('Rock rests at position: ' + str((currX, currY)))
            break
    
    place_rock(cave, currRock, currX, currY)
    
    # recalculate cave height
    h = cave.shape[0] - 1
    while cave[h,:].max() < 1:
        h -= 1
    heightRocks = h
    
    nbRocks += 1


In [9]:
print(heightRocks)


3068
